In [1]:
import pandas as pd

artigo_treino = pd.read_csv("treino.csv")
artigo_teste = pd.read_csv("teste.csv")
artigo_treino.sample()

,title,text,date,category,subcategory,link
39209,Mascarados distribuem panfletos com ameaças a ...,Uma gangue de homens mascarados e vestidos de ...,2016-01-30,mundo,NaN,http://www1.folha.uol.com.br/mundo/2016/01/173...


## Count Vectorizer

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
texto = ["tenha um bom dia",
         "tenha um péssimo dia",
         "tenha um ótimo dia",
         "tenha um dia ruim"]

vetorizador = CountVectorizer()
vetorizador.fit(texto)

CountVectorizer()

In [4]:
print(vetorizador.vocabulary_)

{'tenha': 4, 'um': 5, 'bom': 0, 'dia': 1, 'péssimo': 2, 'ótimo': 6, 'ruim': 3}


In [5]:
vetor_bom = vetorizador.transform(["bom"])
print(vetor_bom.toarray())

[[1 0 0 0 0 0 0]]


In [6]:
vetor_otimo = vetorizador.transform(["ótimo"])
print(vetor_otimo.toarray())

[[0 0 0 0 0 0 1]]


## Word2Vec

In [7]:
from gensim.models import KeyedVectors

modelo = KeyedVectors.load_word2vec_format("cbow_s300.txt")

In [8]:
modelo.most_similar("china")

[('rússia', 0.7320704460144043),
 ('índia', 0.7241617441177368),
 ('tailândia', 0.7019359469413757),
 ('indonésia', 0.6860769987106323),
 ('turquia', 0.6741335988044739),
 ('malásia', 0.6665689945220947),
 ('mongólia', 0.6593616008758545),
 ('manchúria', 0.6581847071647644),
 ('urss', 0.6581669449806213),
 ('grã-bretanha', 0.6568098068237305)]

In [9]:
modelo.most_similar("itália")

[('frança', 0.79836106300354),
 ('grécia', 0.7547341585159302),
 ('espanha', 0.7417078018188477),
 ('holanda', 0.7343161702156067),
 ('grã-bretanha', 0.7281739711761475),
 ('inglaterra', 0.7172061204910278),
 ('bélgica', 0.7155542373657227),
 ('alemanha', 0.7135661840438843),
 ('suécia', 0.6831519603729248),
 ('áustria', 0.6822685599327087)]

In [10]:
modelo.most_similar(positive=["brasil", "argentina"])

[('chile', 0.6781662702560425),
 ('peru', 0.6348032355308533),
 ('venezuela', 0.6273865103721619),
 ('equador', 0.6037015318870544),
 ('bolívia', 0.6017141342163086),
 ('haiti', 0.599380612373352),
 ('méxico', 0.596230685710907),
 ('paraguai', 0.5957703590393066),
 ('uruguai', 0.5903672575950623),
 ('japão', 0.5893509984016418)]

In [11]:
# nuvem -> nuvens : estrelas -> estrela
# nuvens + estrela - nuvem = estrelas

modelo.most_similar(positive=["nuvens", "estrela"], negative="nuvem")

[('estrelas', 0.5497430562973022),
 ('plêiades', 0.3791978657245636),
 ('colinas', 0.3746805489063263),
 ('trovoadas', 0.37370336055755615),
 ('sombras', 0.37341946363449097),
 ('pombas', 0.3726757764816284),
 ('corredoras', 0.3640727400779724),
 ('cigarras', 0.36065393686294556),
 ('galáxias', 0.35754910111427307),
 ('luas', 0.3575345277786255)]

In [12]:
modelo.most_similar(positive=["médico", "mulher"], negative="homem")

[('enfermeira', 0.6180862188339233),
 ('psicóloga', 0.47447535395622253),
 ('dama-de-companhia', 0.47382354736328125),
 ('esposa', 0.4655913710594177),
 ('parteira', 0.4636286795139313),
 ('mãe', 0.45817673206329346),
 ('governanta', 0.45722928643226624),
 ('madrasta', 0.4569782614707947),
 ('menina', 0.44435903429985046),
 ('filha', 0.44341105222702026)]

In [13]:
modelo.most_similar(positive=["professor", "mulher"], negative="homem")

[('professora', 0.6192207932472229),
 ('aluna', 0.5449554324150085),
 ('esposa', 0.4978231191635132),
 ('ex-aluna', 0.4884248375892639),
 ('namorada', 0.4737859070301056),
 ('enfermeira', 0.4728143811225891),
 ('filha', 0.46737387776374817),
 ('irmã', 0.4584592282772064),
 ('ex-namorada', 0.45824769139289856),
 ('ex-professora', 0.4510471522808075)]

## Vetorização de texto

In [14]:
artigo_treino.title.loc[12]

"Daniel Craig será stormtrooper em novo 'Star Wars', diz ator"

In [17]:
import nltk
import string
nltk.download("punkt")

def tokenizador(texto):
    texto = texto.lower()
    lista_alfanumerica = []

    for token_valido in nltk.word_tokenize(texto):
        if token_valido in string.punctuation: continue
        lista_alfanumerica.append(token_valido)

    return lista_alfanumerica

tokenizador("Texto Exemplo, 1234.")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pauloceliojr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['texto', 'exemplo', '1234']

In [26]:
import numpy as np

# Combinação de vetores por soma
def combinacao_vetores(palavras_numeros):
    vetor_resultante = np.zeros(300)

    for pn in palavras_numeros:
        try:
            vetor_resultante += modelo.get_vector(pn)
        except KeyError:
            if pn.isnumeric():
                pn = "0" * len(pn) # "0, 00, 000, 0000, etc. dependendo do tamanho de caracteres do número
                vetor_resultante += modelo.get_vector(pn)
            else:
                vetor_resultante += modelo.get_vector("unknown")

    return vetor_resultante

In [30]:
palavras_numeros = tokenizador("texto caelumx")
vetor_texto = combinacao_vetores(palavras_numeros)
print(len(vetor_texto))
print(vetor_texto)

300
[ 4.04902007e-01  1.61033005e-01 -1.28350001e-01  4.63957988e-01
 -3.79070006e-02  9.79099981e-02  6.06219992e-02  2.50669867e-02
 -1.45315003e-01 -1.20187001e-01 -1.45029984e-02 -3.97820026e-02
 -5.53439990e-01  3.39491010e-01  4.74720001e-02  2.32610006e-01
 -7.91507989e-01  7.62100518e-03 -4.30413000e-01 -1.62210986e-01
 -1.36736006e-01 -2.92862996e-01 -7.81050026e-02  4.56106991e-01
 -1.08597010e-01 -1.72903001e-01 -3.94289017e-01 -7.94299692e-03
  1.54560007e-01 -1.81829929e-02 -1.44808993e-01 -1.92361996e-01
  5.27470000e-02  2.77960012e-02  3.75599414e-03  2.15162002e-01
  2.41512008e-01  2.07674988e-01  1.77449007e-01  2.00845003e-01
  1.08239995e-02 -1.03557003e-01  4.52470034e-02  3.54460012e-02
  2.74897987e-01 -7.71570001e-02  4.26173002e-01  4.57773015e-01
  2.98846003e-01  2.85166003e-01  1.75556996e-01  1.93372004e-01
 -2.64183000e-01  1.87287997e-01  2.00636007e-01 -1.48244001e-01
  1.16369996e-01  5.01876995e-01 -2.73249969e-02 -3.36082000e-01
 -1.06219009e-01  4.2

In [32]:
def matriz_vetores(textos):
    x = len(textos)
    y = 300 # número de dimensões do dicionário/vocabulário cbow300
    matriz = np.zeros((x, y))

    for i in range(x):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_vetores(palavras_numeros)

    return matriz

matriz_vetores_treino = matriz_vetores(artigo_treino.title)
matriz_vetores_teste = matriz_vetores(artigo_teste.title)
print(matriz_vetores_treino.shape)
print(matriz_vetores_teste.shape)

(90000, 300)
(20513, 300)


In [36]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=200)
lr.fit(matriz_vetores_treino, artigo_treino.category)

LogisticRegression(max_iter=200)

In [38]:
lr.score(matriz_vetores_teste, artigo_teste.category)

0.7957880368546775

In [39]:
artigo_teste.category.unique()

array(['colunas', 'esporte', 'mercado', 'cotidiano', 'mundo', 'ilustrada'],
      dtype=object)

In [41]:
from sklearn.metrics import classification_report

label_prevista = lr.predict(matriz_vetores_teste)
cr = classification_report(artigo_teste.category, label_prevista)
print(cr)

              precision    recall  f1-score   support

     colunas       0.86      0.71      0.78      6103
   cotidiano       0.61      0.79      0.69      1698
     esporte       0.92      0.88      0.90      4663
   ilustrada       0.13      0.88      0.23       131
     mercado       0.84      0.79      0.81      5867
       mundo       0.74      0.86      0.79      2051

    accuracy                           0.80     20513
   macro avg       0.68      0.82      0.70     20513
weighted avg       0.83      0.80      0.81     20513



In [44]:
from sklearn.dummy import DummyClassifier
dc = DummyClassifier()
dc.fit(matriz_vetores_treino, artigo_treino.category)
label_prevista_dc = dc.predict(matriz_vetores_teste)
cr_dummy = classification_report(artigo_teste.category, label_prevista_dc)
print(cr_dummy)

C:\Users\pauloceliojr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     colunas       0.30      1.00      0.46      6103
   cotidiano       0.00      0.00      0.00      1698
     esporte       0.00      0.00      0.00      4663
   ilustrada       0.00      0.00      0.00       131
     mercado       0.00      0.00      0.00      5867
       mundo       0.00      0.00      0.00      2051

    accuracy                           0.30     20513
   macro avg       0.05      0.17      0.08     20513
weighted avg       0.09      0.30      0.14     20513



C:\Users\pauloceliojr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pauloceliojr\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
